# 기본세팅

In [ ]:
import numpy as np
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import pandas as pd 
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# Change parameter
 이 부분만 변환하고 나머지는 건들지 않습니다.

In [ ]:
'''
================================================================================================
'서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
'세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
'경상남도', '제주특별자치도'
================================================================================================
'''

DO_list = ['세종특별자치시'] # 수집하고자 하는 지역

# 크롤링

In [ ]:
categorys = ['pc방', '노래방', '보드게임카페'] ### 지도에 검색하고자 하는 키워드 입력

In [ ]:
t1 = time.time()

for DO in DO_list:
    
    # 데이터프레임 초기화
    result = pd.DataFrame(columns = ['id','가맹점명','naver_category','전화번호','lat','lng','도로명주소'])
    hangjeong = pd.read_excel('세종시_행정동명.xlsx', engine='openpyxl')
    hangjeong_detail = hangjeong.loc[hangjeong['도'] == DO, ['시군', '읍면동구']].reset_index(drop=True)

    # 행정동을 위한 for문
    region_num_count = 0
    print('=== 지역명: %s === 전체 지역의 개수: %d ===' %(DO, len(hangjeong_detail)))

    for SIGUN, GU in zip(hangjeong_detail['시군'], hangjeong_detail['읍면동구']):
        region_num_count += 1
        print('== %d번째 지역인 %s 지역시작 ==' %(region_num_count, SIGUN+' '+GU))
        driver.quit()   # 크롬 메모리 초기화
        driver = webdriver.Chrome('chromedriver_91.exe', options=options)

        # 해당 행정동 내 모든 카테고리를 돌기위한 for문
        count = 0    
        for category in categorys:
            count += 1
            t2 = time.time()
            elapsed_time2 = round(t2-t1)
            print('%d번째 %s 카테고리 시작 === 경과시간: %d초' % (count, category, elapsed_time2))

            # 크롬 메모리 초기화
            if count == 150:
                driver.quit()   
                driver = webdriver.Chrome('chromedriver_91.exe', options=options)

            driver.get('https://m.place.naver.com/place/list?query={}%20{}%20{}'.format(SIGUN ,GU, category))
            time.sleep(7)
            
            # 처음 
            if count == 1:
                time.sleep(5)

            # 목록보기 누르기
            try:
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div/a').click()
            except:
                time.sleep(2)
                driver.find_element_by_xpath('//*[@id="app-root"]/div/div/a').click()


            # 스크롤을 최하단으로
            while True:   
                try:
                    elements = driver.find_elements_by_class_name("_16f7Q")
                    last = driver.find_elements_by_class_name("_16f7Q")[-1]
                    last_name = last.text
                    Len_1 = len(elements) # 초기 항목 합산
                    driver.execute_script("arguments[0].scrollIntoView();", last)
                    time.sleep(2)
                    Len_2 = len(driver.find_elements_by_class_name("_16f7Q")) # 스크롤 내린 후 개수 체크

                    if Len_1 == Len_2 :  # 항목이 더이상 없을 경우
                        break
                    if Len_1+1 == Len_2:
                        break

                except Exception as e: 
                    break


            # 정보추출 크롤링
            time.sleep(1)
            elements = driver.find_elements_by_class_name("_1gsZf")  #id 파싱 후 적재
            for element in elements:

                try:
                    value1 = element.find_elements_by_class_name("_1uXIN")[0].find_element_by_tag_name('a').get_attribute('href').split('/')[-1].split('?')[0] #id
                except:
                    value1 = ''

                try:
                    value2 = element.find_elements_by_class_name('_16f7Q')[0].text # 가맹점명
                except:
                    value2 = ''

                try:
                    value3 = element.find_elements_by_class_name('kAdc3')[0].text # 카테고리
                except:
                    value3 = ''

                try:
                    value5 = element.find_element_by_id('_vsl.share').get_attribute("data-line-description") # 도로명주소
                except:
                    value5 = ''

                try:
                    if len(element.find_elements_by_class_name('_3hFgU')) == 3:
                        value6 = element.find_elements_by_class_name('_3hFgU')[0].find_element_by_tag_name('a').get_attribute("href").split(':')[1] # 전화번호
                    else:
                        value6 = ''
                except:
                    value6 = ''

                try:
                    if len(element.find_elements_by_class_name('_3hFgU')) == 3: #lat
                        value7 = element.find_elements_by_class_name('_3hFgU')[1].find_element_by_tag_name('a').get_attribute("href").split('longitude%5E')[2].split('%')[0] # 큰값
                    elif len(element.find_elements_by_class_name('_3hFgU')) == 2:
                        value7 = element.find_elements_by_class_name('_3hFgU')[0].find_element_by_tag_name('a').get_attribute("href").split('longitude%5E')[2].split('%')[0] # 큰값   
                except:
                    value7 = ''

                try:
                    if len(element.find_elements_by_class_name('_3hFgU')) == 3: # lng
                        value8 = element.find_elements_by_class_name('_3hFgU')[1].find_element_by_tag_name('a').get_attribute("href").split('latitude%5E')[2].split('%')[0] # 작은값
                    elif len(element.find_elements_by_class_name('_3hFgU')) == 2:
                        value8 = element.find_elements_by_class_name('_3hFgU')[0].find_element_by_tag_name('a').get_attribute("href").split('latitude%5E')[2].split('%')[0] # 작은값
                except:
                    value8 = ''


                value_df = pd.DataFrame({'id':[value1],
                                '가맹점명':[value2],
                                'naver_category':[value3],
                                '전화번호':[value6],
                                'lat':[value7],
                                'lng':[value8],
                                '도로명주소':[value5]})

                result = result.append(value_df, ignore_index=True)         

        t3 = time.time()
        elapsed_time3 = round(t3-t1)
        print('%s 완료 === 경과시간: %d초' % (SIGUN+' '+GU, elapsed_time3))
    print('크롤링 완료!!!!')

result_final = result.drop_duplicates(subset=['id', '가맹점명']).reset_index(drop=True)
result_final